**Problem statement**: To build a CNN based model which can accurately detect melanoma. Melanoma is a type of cancer that can be deadly if not detected early. It accounts for 75% of skin cancer deaths. A solution which can evaluate images and alert the dermatologists about the presence of melanoma has the potential to reduce a lot of manual effort needed in diagnosis.

**Data Summary:**

The dataset consists of 2357 images of malignant and benign oncological diseases, which were formed from the International Skin Imaging Collaboration (ISIC). All images were sorted according to the classification taken with ISIC, and all subsets were divided into the same number of images, with the exception of melanomas and moles, whose images are slightly dominant.

The data set contains the following diseases:
1. Actinic keratosis
2. Basal cell carcinoma
3. Dermatofibroma
4. Melanoma
5. Nevus
6. Pigmented benign keratosis
7. Seborrheic keratosis
8. Squamous cell carcinoma
9. Vascular lesion
 

### Importing all the important libraries

In [ ]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

# Data Reading And Data Understanding

In [ ]:
# Defining the path for train and test images
## Todo: Update the paths of the train and test dataset
data_dir_train = pathlib.Path("Train")
data_dir_test = pathlib.Path('Test')

This assignment uses a dataset of about 2357 images of skin cancer types. The dataset contains 9 sub-directories in each train and test subdirectories. The 9 sub-directories contains the images of 9 skin cancer types respectively.

In [ ]:
image_count_train = len(list(data_dir_train.glob('*/*.jpg')))
print(image_count_train)
image_count_test = len(list(data_dir_test.glob('*/*.jpg')))
print(image_count_test)

# Dataset Creation

### Created train & validation dataset from the train directory with a batch size of 32 .Resized images to 180*180.

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

Use 80% of the images for training, and 20% for validation.

In [ ]:
## Creating Training Dataset
## Used seed=123 while creating your dataset using tf.keras.preprocessing.image_dataset_from_directory
## Resized images to the size img_height*img_width, while writting the dataset

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
  

In [ ]:
## Creating Validation Dataset
## Used seed=123 while creating your dataset using tf.keras.preprocessing.image_dataset_from_directory
## Resized images to the size img_height*img_width, while writting the dataset

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
print(train_ds)

In [ ]:
## Creating Test Dataset
## Used seed=123 while creating your dataset using tf.keras.preprocessing.image_dataset_from_directory
## Resized images to the size img_height*img_width, while writting the dataset

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# List out all the classes of skin cancer and store them in a list. 
# You can find the class names in the class_names attribute on these datasets. 
# These correspond to the directory names in alphabetical order.
class_names = train_ds.class_names
print(class_names)

# Dataset Visualisation
###  Created a code to visualize one instance of all the nine classes present in the dataset 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch,labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break
  

The `image_batch` is a tensor of the shape `(32, 180, 180, 3)`. This is a batch of 32 images of shape `180x180x3` (the last dimension refers to color channels RGB). The `label_batch` is a tensor of the shape `(32,)`, these are corresponding labels to the 32 images.

`Dataset.cache()` keeps the images in memory after they're loaded off disk during the first epoch.

`Dataset.prefetch()` overlaps data preprocessing and model execution while training.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Model Building & training
### Created a CNN model, which can accurately detect 9 classes present in the dataset.
### Defined appropriate optimiser and loss function for model training.
### Trained the model for ~20 epochs.
### Explained the findings after the model fit with evidence if the model overfits or underfits.
####  Use ```layers.experimental.preprocessing.Rescaling``` to normalize pixel values between (0,1). The RGB channel values are in the `[0, 255]` range. This is not ideal for a neural network. Here, it is good to standardize values to be in the `[0, 1]`

In [ ]:
## your code goes here
normalization = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
num_classes = 9
model = Sequential([layers.experimental.preprocessing.Rescaling(1./255,input_shape=(img_height,img_width,3)),
                    layers.Conv2D(16,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'),
                    layers.MaxPooling2D(pool_size=(2,2)),
                    layers.Conv2D(32,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'),
                    layers.MaxPooling2D(pool_size=(2,2)),
                    layers.Conv2D(64,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'),
                    layers.MaxPooling2D(pool_size=(2,2)),
                    layers.Flatten(),
                    layers.Dense(128,activation='softmax'),
                    layers.Dense(num_classes)])

In [ ]:
### Builing a Model
num_classes=9

model = Sequential()
#Adding a  preprocessing_layers to rescale the inputs
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height,img_width,3)))
model.add(Conv2D(16,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='softmax'))
model.add(Dense(num_classes))


## Number of classes is 9
model.summary()

### Compile the model
Choose an appropirate optimiser and loss function for model training 

In [ ]:
# Selected an appropirate optimiser 'adam' and loss function 'SparseCategoricalCrossentropy'

optimizer = 'adam'
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

### Train the model

In [ ]:
#Train the model with 20 epochs and batch size =32.
epochs = 20
batch_size = 32

history = model.fit(
  train_ds,steps_per_epoch=2,
  batch_size=batch_size,
  validation_data=val_ds,
  epochs=epochs
)

### Visualizing training results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


#### Model Overfit or Underfit?

In [ ]:
#checking the model accuracy and loss on train and validation dataset
loss, accuracy = model.evaluate(train_ds, verbose=1,)
loss_v, accuracy_v = model.evaluate(val_ds, verbose=1)

print("Accuracy: ", accuracy)
print("Validation Accuracy: ",accuracy_v)
print("Loss: ",loss)
print("Validation Loss", loss_v)

### findings
#### There is difference of more than 10 percent between training and validation accuracy. Model is Overfitting and we need to chose right data augumentation strategy.

# Model with dropout(),BatchNormalization() preprocessed with ImageDataGenerator

### Data Augmentation

In [ ]:
#Data Augmentaion using ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

image_class = ['nevus','melanoma','basal_cell_caricoma','actinic_keratosis','vasc_lesion','dermatofibroma', 'pigmented_keratosis', 'seborrheic_keratosis', 'squamous_carci']

train_batches = datagen.flow_from_directory(data_dir_train, 
    target_size = (180,180),
    classes = image_class,
    batch_size = 64
 )

valid_batches = datagen.flow_from_directory(data_dir_test, 
    target_size = (180,180),
    classes = image_class,
    batch_size = 64
)


In [ ]:
# visualize how your augmentation strategy works for one instance of training image.
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 20))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:

model_1 = Sequential()
#Adding a  preprocessing_layers to rescale the inputs
model_1.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height,img_width,3)))
model_1.add(Conv2D(16,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'))
model_1.add(MaxPool2D(pool_size=(2, 2)))
# Adding Dropout Layer
model_1.add(Dropout(0.25))

model_1.add(Conv2D(32,kernel_size=(3,3),padding='same',strides=(1,1),activation='relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPool2D(pool_size=(2, 2)))
model_1.add(Dropout(0.4))

model_1.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=(2,2),activation='relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPool2D(pool_size=(2, 2)))
model_1.add(Dropout(0.4))

model_1.add(Flatten())

model_1.add(Dense(128, activation='softmax'))
model_1.add(Dense(num_classes))


## Number of classes is 9
model_1.summary()

### Compiling the model

In [ ]:
## Selected an appropirate optimiser 'adam' and loss function 'categorical_crossentropy'
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model_1.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

I have used an annealing method of the learning rate (LR) in order to make the optimizer converge faster and closest to the global minimum of the loss function, 

The LR is the step by which the optimizer walks through the 'loss landscape'. The higher LR, the bigger are the steps and the quicker is the convergence. However the sampling is very poor with an high LR and the optimizer could probably fall into a local minima.

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.00001)

### Training the model

In [ ]:
#train your model for 20 epochs
epochs = 20
batch_size = 10
history = model_1.fit(train_batches,
                     epochs = epochs,
                      steps_per_epoch=5,
                      verbose = 1, 
                      validation_data=valid_batches, 
                      callbacks=[learning_rate_reduction])

### Visualizing the results

In [ ]:
acc = history.history['accuracy']
print(history.history.keys, ":")
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#checking the model accuracy and loss on train and validation dataset
loss, accuracy = model_1.evaluate(train_batches, verbose=1,)
loss_v, accuracy_v = model_1.evaluate(valid_batches, verbose=1)

print("Accuracy: ", accuracy)
print("Validation Accuracy: ",accuracy_v)
print("Loss: ",loss)
print("Validation Loss", loss_v)

### Findings
#### Model is overfitting as there is substantial difference between Training and validation dataset accuracy.

## Class distribution

In [ ]:
#Plotting the length of the 9 classess of skin disease
import matplotlib.pyplot as plt
data = dict()

for i in class_names:
  data[i] = []

plt.figure(figsize=(5, 5))
for images, labels in train_ds:
  for i in range(9):
    data[class_names[labels[i]]].append(images[i].numpy().astype("uint8"))

for i in data:
  data[i] = len(data[i]) 

f = plt.figure()
f.set_figwidth(20)
f.set_figheight(20)

plt.bar(range(len(data)), list(data.values()), align='center')
plt.xticks(range(len(data)), list(data.keys()))
plt.xticks(rotation = 25)
plt.show()




#### - Which class has the least number of samples? 
**seborrheic keratosis**
#### - Which classes dominate the data in terms proportionate number of samples?
**pigmented benign keratosis** dominates the data of count more than 100 in training


# Managing the Overfit 

### Selected Python package 'Augmentor' an appropriate data augmentation strategy to resolve underfitting/overfitting 

In [ ]:
# importing the libraray Augmentor
import Augmentor
for i in class_names:
    csp = str(data_dir_train)
    fullpath = csp + "/" + i
    p = Augmentor.Pipeline(fullpath)
    p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    p.sample(500) ## We are adding 500 samples per class to make sure that none of the classes are sparse.

Augmentor has stored the augmented images in the output sub-directory of each of the sub-directories of skin cancer types.. Lets take a look at total count of augmented images.

In [ ]:
#Printing the total count of image
image_count_train = len(list(data_dir_train.glob('*/output/*.jpg')))
print(image_count_train)

### Lets see the distribution of augmented data after adding new images to the original training data.

In [ ]:
#Creating a new folder 'Output' 
path_list = [x for x in glob(os.path.join(data_dir_train, '*','output', '*.jpg'))]
path_list

In [ ]:
lesion_list_new = [os.path.basename(os.path.dirname(os.path.dirname(y))) for y in glob(os.path.join(data_dir_train, '*','output', '*.jpg'))]
lesion_list_new

In [ ]:

dataframe_dict_new = dict(zip(path_list, lesion_list_new))

In [ ]:
df2 = pd.DataFrame(list(dataframe_dict_new.items()),columns = ['Path','Label'])
new_df = df2

In [ ]:
new_df.head()

In [ ]:
new_df['Label'].value_counts()

So, now we have added 500 images to all the classes to maintain some class balance. We can add more images as we want to improve training process.

#### Create a training dataset

In [ ]:
batch_size = 32
img_height = 180
img_width = 180
#data_dir_train="/content/gdrive/MyDrive/CNN/Skin_Data_Store/Train"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  seed=123,
  validation_split = 0.2,
  subset = 'training',
  image_size=(img_height, img_width),
  batch_size=batch_size)

#### Create a validation dataset

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  seed=123,
  validation_split = 0.2,
  subset = 'validation',
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Model Building after data augmentation using library Augmentor
#### Create your model (make sure to include normalization)

In [ ]:
model_2 = Sequential()
#Adding a  preprocessing_layers to rescale the inputs
model_2.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height,img_width,3)))
model_2.add(Conv2D(16,kernel_size=(3,3),padding='same',strides=(2,2),activation='relu'))
model_2.add(MaxPool2D(pool_size=(2, 2)))
# Adding Dropout Layer
model_2.add(Dropout(0.25))

model_2.add(Conv2D(32,kernel_size=(3,3),padding='same',strides=(2,2),activation='relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPool2D(pool_size=(2, 2)))
model_2.add(Dropout(0.4))

model_2.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=(2,2),activation='relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPool2D(pool_size=(2, 2)))
model_2.add(Dropout(0.4))

model_2.add(Flatten())

model_2.add(Dense(128, activation='softmax'))
model_2.add(Dense(num_classes))


## Number of classes is 9
model_2.summary()

#### Compile your model (Choose optimizer and loss function appropriately)

In [ ]:
## Selected an appropirate optimiser 'adam' and loss function 'SparseCategoricalCrossentropy'
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

optimizer = 'adam'
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_2.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

####  Train your model

In [ ]:
#Training model on 30 epochs with learning_rate_reductions
epochs = 30

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.00001)

batch_size = 10
history = model_2.fit(train_ds,
                      epochs = epochs,
                      steps_per_epoch =5, 
                      verbose = 1, 
                      validation_data=val_ds, 
                      callbacks=[learning_rate_reduction])

####  Visualize the model results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#checking the model accuracy and loss on train and validation dataset
loss, accuracy = model_2.evaluate(train_ds, verbose=1,)
loss_v, accuracy_v = model_2.evaluate(val_ds, verbose=1)

print("Accuracy: ", accuracy)
print("Validation Accuracy: ",accuracy_v)
print("Loss: ",loss)
print("Validation Loss", loss_v)

### Findings
#### Managing the class imbalance has helped in reducing overfit of the data.Loss is reduced but,  the Acurracy has gone down substantially

#### Did you get rid of underfitting/overfitting? Did class rebalance help?



Managing the class imbalance has helped in reducing overfit of the data.Loss is reduced
But  the Acurracy has gone down substantially


Initially we tried without the ImageDataGenerator which lead data to over fit

Then we introduced dropout,batch normalisation and ImageDataGenerator which increased  the over fit

Finallly, we tried manage  the data imbalance with Augumentor which really helped in reducing the overfit considerbaly

# Test Data Set

In [ ]:
#Evaluating the Model_2 on test dataset
loss_test, accuracy_test = model_2.evaluate(test_ds, verbose=1,)


print("Accuracy: ", accuracy_test)

print("Loss: ",loss_test)


### Findings 

#### Accuracy and Loss of the test Data Set  is quite similar to the Train Data set.